### Multiagent Active Blockference

This notebook is an experimental exploration of multi-agent active inference. CadCAD is not used at this point.

We are considering an environment with two agents, Karl and Thomas, who are trying to move to a preferred state without colliding.

In [41]:
import itertools
import numpy as np
import copy
import sys
from pymdp import utils
import pandas as pd

# adding tools to the system path
sys.path.insert(0, '../../')

from blockference.envs.grid_env import GridAgent
from blockference.gridference import ActiveGridference
from blockference.agent import Agent

In [2]:
# start with 2x2 grid
grid = list(itertools.product(range(3), repeat=2))
border = np.sqrt(len(grid)) - 1
pos_dict = {}
for i in range(0, len(grid)):
    pos_dict[i] = grid[i]
print(pos_dict)
num_agents = 2 # start with 2 agents
init_pos = [0, 3] # agents will start at positions 0 and 3

{0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (1, 0), 4: (1, 1), 5: (1, 2), 6: (2, 0), 7: (2, 1), 8: (2, 2)}


In [3]:
grid_dims = [3, 3] # dimensions of the grid (number of rows, number of columns)
num_grid_points = np.prod(grid_dims) # total number of grid locations (rows X columns)

# create a look-up table `loc_list` that maps linear indices to tuples of (y, x) coordinates 
grid_ = np.arange(num_grid_points).reshape(grid_dims)
it = np.nditer(grid_, flags=["multi_index"])

loc_list = []
while not it.finished:
    loc_list.append(it.multi_index)
    it.iternext()

In [4]:
E = ["UP", "DOWN", "LEFT", "RIGHT", "STAY"]

In [5]:
loc_list

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [6]:
# getting the grid positions and indexes for the two agents K & T
init_K = init_pos[0]
init_T = init_pos[1]
init_K_pos = pos_dict[init_K]
init_T_pos = pos_dict[init_T]

In [7]:
# getting the preferred grid positions and indexes for the two agents A & B
# their preferred position will be the one where the other agent starts
pref_K = 8
pref_T = 0
pref_K_pos = pos_dict[pref_K]
pref_T_pos = pos_dict[pref_T]

#### Observations and States
In a single-agent environment, observations and states are both just the number of positions (because the agent can be at 4 different positions (4 states) and have 4 different observations).

Adding an extra agents adds extra complexity. We let our agents be strictly non-interacting, i.e. they cannot occupy the same position on the grid at the same time.

#### Generative Model Tensors
Now we define the tensors describing the generative model. For detailed explanations of what each tensor means/does, see:
https://pymdp-rtd.readthedocs.io/en/latest/notebooks/active_inference_from_scratch.html

## Alternative way of thinking about states & state modalities (*A1*)
The two modalities of the multiagent POMDP:
- location: "where am I in the world (on the grid)"
- agent awareness: "where is the other agent with respect to me in the world"

These modalities will be reflected in the **A** and **B** matrices.

In [8]:
# location
n_states = len(grid)
n_observations = len(grid)

In [9]:
# A matrix
# Note: maybe multi-agent actinf does not change B but rather A & D
A = np.eye(n_observations, n_states)
print(A)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### Second A modalities

#### Modality 2: absolute pos

In [10]:
A_second = np.eye(n_observations, n_states)
print(A_second)

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


#### Modality 3: relative pos

In [11]:
# other agent relative location (currently 1-step depth)
second_agent_locations = ["NONE", "NEXT_LEFT", "NEXT_RIGHT", "ABOVE", "BELOW"]

n_states_second = len(second_agent_locations)
n_observations_second = len(second_agent_locations)

A_third = np.eye(n_observations_second, n_states_second)
print(A_third)

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [12]:
full_A = np.array([A, A_second, A_third], dtype='object')

end A initialization

In [13]:
full_A

array([array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 1., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 1., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 1., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 1., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 1., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 1., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 1., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
       array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 1., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 1., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 1., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 1., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 1., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 1., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 1., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
       array([[1., 0., 0., 0., 0.],
              [0., 1., 0

In [14]:
# B matrix
# Note: B should only encode prior beliefs about !controllable! transitions between hidden states
# why/how can we assume the actions of the other agents are within controllable transitions?
B = np.zeros((len(grid), len(grid), len(E)))

for action_id, action_label in enumerate(E):

    for curr_state, grid_location in enumerate(grid):

        y, x = grid_location

        if action_label == "DOWN":
            next_y = y - 1 if y > 0 else y
            next_x = x
        elif action_label == "UP":
            next_y = y + 1 if y < border else y
            next_x = x
        elif action_label == "LEFT":
            next_x = x - 1 if x > 0 else x
            next_y = y
        elif action_label == "RIGHT":
            next_x = x + 1 if x < border else x
            next_y = y
        elif action_label == "STAY":
            next_x = x
            next_y = y
        new_location = (next_y, next_x)
        next_state = grid.index(new_location)
        B[next_state, curr_state, action_id] = 1.0
print(B)

[[[0. 1. 1. 0. 1.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0.]
  [0. 1. 0. 0. 1.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 1. 0. 1. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 1.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0.]
  

Second modality of the **B** matrix is the transition probabilities given an observation of a second agent.
This can either be:
- "there is an agent *above* me"
- "there is an agent *below* me"
- "there is an agent *to the right* of me"
- "there is an agent *to the left* of me"
- "there is no agent next to me

This modality should track the *relative* position of the agent with respect to the second agent.
This can then be scaled to arbitrary many agents by using this matrix for tracking the position of different agents relative to each other.

In the following, the K_agent is the one whose generative model we're modeling, T_agent is the agent who K_agent is perceiving.

(Note: we might possibly need to add a third modality, colliding/not-colliding, for encoding preferences)

In [15]:
second_agent_locations = ["NONE", "NEXT_LEFT", "NEXT_RIGHT", "ABOVE", "BELOW"]

B_second = np.zeros((len(second_agent_locations), len(second_agent_locations), len(E)))
pos_idx = {"NONE": 0, "NEXT_LEFT": 1, "NEXT_RIGHT": 2, "ABOVE": 3, "BELOW": 4}

for action_id, action_label in enumerate(E):

    for curr_state, T_location in enumerate(second_agent_locations):

        if action_label == "UP":
            next_T_location = "NONE" if T_location != "ABOVE" else "ABOVE"
        elif action_label == "DOWN":
            next_T_location = "NONE" if T_location != "BELOW" else "BELOW"
        elif action_label == "LEFT":
            next_T_location = "NONE" if T_location != "NEXT_LEFT" else "NEXT_LEFT"
        elif action_label == "RIGHT":
            next_T_location = "NONE" if T_location != "NEXT_RIGHT" else "NEXT_RIGHT"
        elif action_label == "STAY":
            next_T_location = T_location
        new_T_location = next_T_location
        next_state = pos_idx[new_T_location]
        B_second[next_state, curr_state, action_id] = 1.0

print(B_second)

[[[1. 1. 1. 1. 1.]
  [1. 1. 0. 1. 0.]
  [1. 1. 1. 0. 0.]
  [0. 1. 1. 1. 0.]
  [1. 0. 1. 1. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 1.]]]


In [16]:
full_B = np.array([B, B_second], dtype='object')

End of B initialization

In [17]:
full_B.shape

(2,)

In [18]:
A_gm = copy.deepcopy(full_A)
B_gm = copy.deepcopy(full_B)

In [19]:
A_gm

array([array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 1., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 1., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 1., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 1., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 1., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 1., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 1., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
       array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
              [0., 1., 0., 0., 0., 0., 0., 0., 0.],
              [0., 0., 1., 0., 0., 0., 0., 0., 0.],
              [0., 0., 0., 1., 0., 0., 0., 0., 0.],
              [0., 0., 0., 0., 1., 0., 0., 0., 0.],
              [0., 0., 0., 0., 0., 1., 0., 0., 0.],
              [0., 0., 0., 0., 0., 0., 1., 0., 0.],
              [0., 0., 0., 0., 0., 0., 0., 1., 0.],
              [0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
       array([[1., 0., 0., 0., 0.],
              [0., 1., 0

## Another alternate approach (*A2*)
(trying to fix the A & B tensors based on epistemic chaining tutorial in pymdp docs)

In [8]:
# second_agent_rel_locations = ["NONE", "NEXT_LEFT", "NEXT_RIGHT", "ABOVE", "BELOW"]
second_agent_locations = len(grid)

In [9]:
num_obs = [num_grid_points, second_agent_locations]

In [10]:
num_states = [num_grid_points]

In [11]:
# the preference array
C = utils.obj_array_zeros(num_obs)

In [12]:
# the prior belief array
D = utils.obj_array_uniform(num_states)

The observation model: the **A** array

In [13]:
A_m_shapes = [ [o_dim] + num_states for o_dim in num_obs] # list of shapes of modality-specific A[m] arrays
A = utils.obj_array_zeros(A_m_shapes) # initialize A array to an object array of all-zero subarrays

In [14]:
# make the location observation only depend on the location state (proprioceptive observation modality)
A[0] = np.eye(num_grid_points)

In [15]:
# The other agent location is independent of the reference agent location
A[1] = copy.deepcopy(A[0])

In [16]:
A[0]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]])

The transition model: the **B** array

In [17]:
num_controls = [5, 1] # 5 movement affordances, none controls for the location of the other agent

# initialize the shapes of each sub-array `B[f]`
B_f_shapes = [ [ns, ns, num_controls[f]] for f, ns in enumerate(num_states)]

# create the `B` array and fill it out
B = utils.obj_array_zeros(B_f_shapes)

In [18]:
actions = ["UP", "DOWN", "LEFT", "RIGHT", "STAY"]

# fill out `B[0]` using the 
for action_id, action_label in enumerate(actions):

  for curr_state, grid_location in enumerate(loc_list):

    y, x = grid_location

    if action_label == "UP":
      next_y = y - 1 if y > 0 else y 
      next_x = x
    elif action_label == "DOWN":
      next_y = y + 1 if y < (grid_dims[0]-1) else y 
      next_x = x
    elif action_label == "LEFT":
      next_x = x - 1 if x > 0 else x 
      next_y = y
    elif action_label == "RIGHT":
      next_x = x + 1 if x < (grid_dims[1]-1) else x 
      next_y = y
    elif action_label == "STAY":
      next_x = x
      next_y = y

    new_location = (next_y, next_x)
    next_state = loc_list.index(new_location)
    B[0][next_state, curr_state, action_id] = 1.0

## Building the cadCAD simulation

In [19]:
# controllable_indices = [0, 1]
controllable_indices = [0]
# controllable_indices = [1]

In [20]:
# agent = Agent(A=full_A, B=B_gm, control_fac_idx=controllable_indices) # A1
agent = Agent(A=A, B=B, C=C, D=D, control_fac_idx=controllable_indices, policy_len=4) #A2

In [21]:
# agent.D = [init_K, 0] # initial K position & initial K position relative to T, 0 means "NONE"
agent.D[0] = utils.onehot(loc_list.index((0,0)), num_grid_points)

In [22]:
# E vector (affordances)
E = ["UP", "DOWN", "LEFT", "RIGHT", "STAY"]

In [23]:
# agent.E = E # adding agent affordances to Agent class instance

In [24]:
# agent.C = [pref_K, 0] # preferred location & preferred relative relation to second agent (again "NONE")

This concludes the initialization of the single agent for the multi-agent POMDP. What follows is an attempt at a full 2-agent Blockference simulation.

In [25]:
# ! pip install radcad --quiet

In [26]:
from radcad import Model, Simulation, Experiment

In [27]:
agent_K = copy.deepcopy(agent)
agent_T = copy.deepcopy(agent)

In [28]:
# change Karl and Thomas' prior & preference
# agent_K.D = [init_K, 0]
# agent_K.C = [pref_K, 0]

# agent_T.D = [init_T, 0]
# agent_T.C = [pref_T, 0]

agent_K.D[0] = utils.onehot(loc_list.index(init_K_pos), num_grid_points)
agent_K.C[0][8] = 1.0
agent_K.C[1][0] = 1.0

agent_T.D[0] = utils.onehot(loc_list.index(init_T_pos), num_grid_points)
agent_T.C[1][8] = 1.0
agent_T.C[0][0] = 1.0

In [73]:
agent.E

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [29]:
# A1
# agent_K.D = np.array((utils.onehot(init_K, len(grid)), utils.onehot(0, len(second_agent_locations))), dtype='object')
# agent_T.D = np.array((utils.onehot(init_T, len(grid)), utils.onehot(0, len(second_agent_locations))), dtype='object')

# A2
agent_K.D = np.array((utils.onehot(init_K, len(grid)), utils.onehot(init_T, len(grid))), dtype='object')
agent_T.D = np.array((utils.onehot(init_T, len(grid)), utils.onehot(init_K, len(grid))), dtype='object')

In [30]:
init_obs_T = [np.nonzero(agent_T.D[0])[0][0], np.nonzero(agent_T.D[1])[0][0]]
init_obs_K = [np.nonzero(agent_K.D[0])[0][0], np.nonzero(agent_K.D[1])[0][0]]

In [31]:
env = GridAgent(grid_len=3, agents=[agent_K, agent_T])

In [44]:
initial_state = {
    'agent_K': agent_K,
    'agent_T': agent_T,
    'env': env,
    'obs': [init_obs_K, init_obs_T],
    'locations': env.agent_locs,
    'actions': [None, None]
}

In [45]:
params = {
}

In [74]:
def p_actinf(params, substep, state_history, previous_state):
    actions = []
    word_actions = []
    for idx, agent in enumerate([previous_state['agent_K'], previous_state['agent_T']]):
        obs = previous_state['obs'][idx] if previous_state['obs'] != '' else agent.D
        qx = agent.infer_states(previous_state['obs'][idx] if previous_state['obs'] != '' else agent.D)
        q_pi, efe = agent.infer_policies()

        action = agent.sample_action()
        word_actions.append(E[int(action)])
        actions.append(action)

    return {'update_actions': actions,
            'update_word_actions': word_actions}

In [75]:
def s_obs(params, substep, state_history, previous_state, policy_input):
    updated_obs = previous_state['env'].step(policy_input['update_actions'])
    return 'obs', updated_obs

def s_act(params, substep, state_history, previous_state, policy_input):
    return 'actions', policy_input['update_word_actions']

In [76]:
state_update_blocks = [
    {
        'policies': {
            'p_actinf': p_actinf
        },
        'variables': {
            'obs': s_obs,
            'actions': s_act
        }
    }
]

In [77]:
model = Model(
    # Model initial state
    initial_state=initial_state,
    # Model Partial State Update Blocks
    state_update_blocks=state_update_blocks,
    # System Parameters
    params=params
)

In [78]:
simulation = Simulation(
    model=model,
    timesteps=20,  # Number of timesteps
    runs=1  # Number of Monte Carlo Runs
)

In [79]:
result = simulation.run()

actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0.])] with length: 2
actions received: [array([0.]), array([0

In [80]:
df = pd.DataFrame(result)
df

,agent_K,agent_T,env,obs,locations,actions,simulation,subset,run,substep,timestep
0,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[[0, 3], [3, 0]]","[0, 3]","[None, None]",0,0,1,0,0
1,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[(3, 1), (6, 1)]","[0, 3]","[UP, UP]",0,0,1,1,1
2,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[(3, 1), (6, 1)]","[0, 3]","[UP, UP]",0,0,1,1,2
3,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[(3, 1), (6, 1)]","[0, 3]","[UP, UP]",0,0,1,1,3
4,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[(3, 1), (6, 1)]","[0, 3]","[UP, UP]",0,0,1,1,4
5,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[(3, 1), (6, 1)]","[0, 3]","[UP, UP]",0,0,1,1,5
6,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[(3, 1), (6, 1)]","[0, 3]","[UP, UP]",0,0,1,1,6
7,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[(3, 1), (6, 1)]","[0, 3]","[UP, UP]",0,0,1,1,7
8,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[(3, 1), (6, 1)]","[0, 3]","[UP, UP]",0,0,1,1,8
9,<blockference.agent.Agent object at 0x1385f0640>,<blockference.agent.Agent object at 0x1385f6280>,<blockference.envs.grid_env.GridAgent object a...,"[(3, 1), (6, 1)]","[0, 3]","[UP, UP]",0,0,1,1,9


## Playground

In [ ]:
test = -qs[factor].dot(prior[factor][:, np.newaxis]) # this is what breaks

In [50]:
num_obs, num_states, num_modalities, num_factors = utils.get_model_dimensions(A = A)

In [56]:
num_states

[9, 1, 9]

In [66]:
A[0][-1][1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [38]:
test_array = np.array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [40]:
test_array.shape

(9,)

In [41]:
np.log(test_array + 1e-16)

array([  0.        , -36.84136149, -36.84136149, -36.84136149,
       -36.84136149, -36.84136149, -36.84136149, -36.84136149,
       -36.84136149])

In [ ]:
np.log

In [36]:
def dot_likelihood(A,obs):

    s = np.ones(np.ndim(A), dtype = int)
    s[0] = obs.shape[0]
    X = A * obs.reshape(tuple(s))
    X = np.sum(X, axis=0, keepdims=True)
    LL = np.squeeze(X)

    # check to see if `LL` is a scalar
    if np.prod(LL.shape) <= 1.0:
        LL = LL.item()
        LL = np.array([LL]).astype("float64")

    return LL

In [ ]:
dot_likelihood()

In [60]:
agent_K.A.shape

(3,)

In [90]:
B

array([array([[[1., 0., 1., 0., 1.],
               [0., 0., 1., 0., 0.],
               [0., 0., 0., 0., 0.],
               [1., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.]],

              [[0., 0., 0., 1., 0.],
               [1., 0., 0., 0., 1.],
               [0., 0., 1., 0., 0.],
               [0., 0., 0., 0., 0.],
               [1., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.]],

              [[0., 0., 0., 0., 0.],
               [0., 0., 0., 1., 0.],
               [1., 0., 0., 1., 1.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [1., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0.

In [56]:
utils.get_model_dimensions(agent_K.A, agent_K.B)

([9, 5], [9, 5], 2, 2)

In [52]:
list(agent.A[0].shape[1:]) if utils.is_obj_array(agent.A) else list(agent.A.shape[1:])

[4, 2]

In [56]:
agent.B.shape

(2,)

In [57]:
agent_K.B.shape

(2,)

In [75]:
utils.get_model_dimensions(A = agent.A)

([4, 3, 2], [4, 2], 3, 2)

In [41]:
num_obs

[9, 5]

In [42]:
num_states

[9]

In [66]:
agent.A

array([array([[[1., 1.],
               [0., 0.],
               [0., 0.],
               [0., 0.]],

              [[0., 0.],
               [1., 1.],
               [0., 0.],
               [0., 0.]],

              [[0., 0.],
               [0., 0.],
               [1., 1.],
               [0., 0.]],

              [[0., 0.],
               [0., 0.],
               [0., 0.],
               [1., 1.]]]), array([[[1.  , 1.  ],
                                    [0.  , 0.  ],
                                    [0.  , 0.  ],
                                    [1.  , 1.  ]],

                                   [[0.  , 0.  ],
                                    [0.98, 0.02],
                                    [0.02, 0.98],
                                    [0.  , 0.  ]],

                                   [[0.  , 0.  ],
                                    [0.02, 0.98],
                                    [0.98, 0.02],
                                    [0.  , 0.  ]]]),
       array

In [82]:
agent_K.A[1][0].shape

(5,)

In [77]:
agent.A[0][0].shape

(4, 2)

In [44]:
# TMAZE
import os
import sys
import pathlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy

from pymdp.agent import Agent
from pymdp import utils
from pymdp.envs import TMazeEnv

In [45]:
def plot_beliefs(belief_dist, title=""):
    plt.grid(zorder=0)
    plt.bar(range(belief_dist.shape[0]), belief_dist, color='r', zorder=3)
    plt.xticks(range(belief_dist.shape[0]))
    plt.title(title)
    plt.show()
    
def plot_likelihood(A, title=""):
    ax = sns.heatmap(A, cmap="OrRd", linewidth=2.5)
    plt.xticks(range(A.shape[1]))
    plt.yticks(range(A.shape[0]))
    plt.title(title)
    plt.show()

In [46]:
reward_probabilities = [0.98, 0.02] # probabilities used in the original SPM T-maze demo
env = TMazeEnv(reward_probs = reward_probabilities)
# here, we can get the likelihood mapping directly from the environmental class. So this is the likelihood mapping that truly describes the relatinoship between the 
# environment's hidden state and the observations the agent will get

A_gp = env.get_likelihood_dist()

In [47]:
A_gp

array([array([[[1., 1.],
               [0., 0.],
               [0., 0.],
               [0., 0.]],

              [[0., 0.],
               [1., 1.],
               [0., 0.],
               [0., 0.]],

              [[0., 0.],
               [0., 0.],
               [1., 1.],
               [0., 0.]],

              [[0., 0.],
               [0., 0.],
               [0., 0.],
               [1., 1.]]]), array([[[1.  , 1.  ],
                                    [0.  , 0.  ],
                                    [0.  , 0.  ],
                                    [1.  , 1.  ]],

                                   [[0.  , 0.  ],
                                    [0.98, 0.02],
                                    [0.02, 0.98],
                                    [0.  , 0.  ]],

                                   [[0.  , 0.  ],
                                    [0.02, 0.98],
                                    [0.98, 0.02],
                                    [0.  , 0.  ]]]),
       array

In [48]:
B_gp = env.get_transition_dist()

In [49]:
A_gm = copy.deepcopy(A_gp) # make a copy of the true observation likelihood to initialize the observation model
B_gm = copy.deepcopy(B_gp) # make a copy of the true transition likelihood to initialize the transition model
controllable_indices = [0] # this is a list of the indices of the hidden state factors that are controllable
agent = Agent(A=A_gm, B=B_gm, control_fac_idx=controllable_indices)
agent.C[1][1] = 3.0
agent.C[1][2] = -3.0

In [50]:
T = 5 # number of timesteps

obs = env.reset() # reset the environment and get an initial observation

# these are useful for displaying read-outs during the loop over time
reward_conditions = ["Right", "Left"]
location_observations = ['CENTER','RIGHT ARM','LEFT ARM','CUE LOCATION']
reward_observations = ['No reward','Reward!','Loss!']
cue_observations = ['Cue Right','Cue Left']
msg = """ === Starting experiment === \n Reward condition: {}, Observation: [{}, {}, {}]"""
print(msg.format(reward_conditions[env.reward_condition], location_observations[obs[0]], reward_observations[obs[1]], cue_observations[obs[2]]))

for t in range(T):
    print(f'Observation is {obs}')
    qx = agent.infer_states(obs)

    q_pi, efe = agent.infer_policies()

    action = agent.sample_action()

    msg = """[Step {}] Action: [Move to {}]"""
    print(msg.format(t, location_observations[int(action[0])]))

    obs = env.step(action)

    msg = """[Step {}] Observation: [{},  {}, {}]"""
    print(msg.format(t, location_observations[obs[0]], reward_observations[obs[1]], cue_observations[obs[2]]))

 === Starting experiment === 
 Reward condition: Left, Observation: [CENTER, No reward, Cue Left]
Observation is [0, 0, 1]
type of num states is <class 'list'>
modality is 0, length of A is 3, A[modality] is [[[1. 1.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [1. 1.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [1. 1.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [1. 1.]]] and obs[modality] is [1. 0. 0. 0.]
ll is [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
dot likelihood is [[1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
modality is 1, length of A is 3, A[modality] is [[[1.   1.  ]
  [0.   0.  ]
  [0.   0.  ]
  [1.   1.  ]]

 [[0.   0.  ]
  [0.98 0.02]
  [0.02 0.98]
  [0.   0.  ]]

 [[0.   0.  ]
  [0.02 0.98]
  [0.98 0.02]
  [0.   0.  ]]] and obs[modality] is [1. 0. 0.]
ll is [[1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
dot likelihood is [[1. 1.]
 [0. 0.]
 [0. 0.]
 [1. 1.]]
modality is 2, length of A is 3, A[modality] is [[[0.5 0.5]
  [0.5 0.5]
  [0.5 0.5]
  [1.  0. ]]

 [[0.5 0.5]
  [0.5 0.5]
  [0.5 0.5]
  [0.